In [7]:
import numpy as np

# import pandas as pd
from implementations import *
import matplotlib.pyplot as plt
from helpers import *
from preprocess import *
from knn import *

In [8]:
y_train, X_train, ids = load_csv_data("data/train.csv")
_, X_test, ids_test = load_csv_data("data/test.csv")

preproc = Preprocessor()
preproc.fit(X_train)
norm_x_train = preproc.standardize(X_train)
norm_x_test = preproc.standardize(X_test)
X_test = knn_random_fill(X_test, norm_x_test, X_train, norm_x_train)
X_train = knn_random_fill(X_train, norm_x_train, X_train, norm_x_train)


ids = ids[0:1000]
# replace empty value -999 by nan.
X_train[X_train == -999] = np.nan
X_test[X_test == -999] = np.nan

size_full_idx 11
2
0 50
[[ 132.049   79.589   23.916 ... -999.    -999.       0.   ]
 [ 106.398   67.49    87.949 ... -999.    -999.      47.575]
 [ 117.794   56.226   96.358 ... -999.    -999.       0.   ]
 ...
 [ 107.052   32.062   68.288 ...   -3.433    2.785  113.525]
 [  95.534   41.53    58.648 ... -999.    -999.     148.942]
 [ 132.049   58.57   111.162 ... -999.    -999.      30.208]]
size_full_idx 15
3
0 50
[[138.47   51.655  97.827 ...   1.24   -2.475 113.497]
 [160.937  68.768 103.235 ...  -2.385   1.876  46.226]
 [120.506 162.172 125.953 ...  -2.385   1.876  44.251]
 ...
 [130.079  53.335  97.435 ...   1.773  -2.079   0.   ]
 [167.534  41.277 123.308 ...  -1.285  -0.995   0.   ]
 [ 95.407  21.584  67.766 ...   1.24   -2.475   0.   ]]


In [9]:
f = open("data/train.csv")
column_names = f.readline().strip().split(",")[2:]
f.close()
# print(column_names)

In [10]:
def jet_num_subset(X, y):
    """
    Based on jet_num, we split the dataset into 3 subsets, with jet_num = 0, 1, and 2 or more.
    """
    zero = X[:, 22] == 0
    one = X[:, 22] == 1
    others = X[:, 22] >= 2

    X_others, y_others = X[others], y[others]
    X_zero, y_zero = X[zero], y[zero]
    X_one, y_one = X[one], y[one]

    return X_zero, y_zero, X_one, y_one, X_others, y_others

In [11]:
# Split test and train datasets by jet numbs.
(
    X_train_zero,
    y_train_zero,
    X_train_one,
    y_train_one,
    X_train_many,
    y_train_many,
) = jet_num_subset(X_train, y_train)
(
    X_test_zero,
    ids_test_zero,
    X_test_one,
    ids_test_one,
    X_test_many,
    ids_test_many,
) = jet_num_subset(X_test, ids_test)

X_train_full = X_train.copy()
X_test_full = X_test.copy()

In [12]:
def combine_f1_scores(values_1, count_1, values_2, count_2, values_3, count_3):
    """
    Merges together F1 score and accuracy on subsets of data
    Arguments: values_i: Triple of the form 1-accuracy,F_1_neg_score, F_1_pos_score as returned by the compute loss function
               count_i: How many elements are in that set
    """
    TP_list = []
    TN_list = []
    FNR_FPR = []
    for (i, count) in zip([values_1, values_2, values_3], [count_1, count_2, count_3]):
        (minus_acc, F_1_pos, F_1_neg) = i
        acc = (1 - minus_acc) * count
        true_negs = F_1_neg * (count - acc) / (2 * (1 - F_1_neg))
        true_pos = acc - true_negs
        fnr_fpr = (2 * true_pos * (1 - F_1_pos)) / F_1_pos
        TP_list.append(true_pos)
        TN_list.append(true_negs)
        FNR_FPR.append(fnr_fpr)
    print(
        "Combined accuracy: ",
        (sum(TP_list) + sum(TN_list)) / (count_1 + count_2 + count_3),
    )
    print(
        "Combined alternative F1: ",
        (2 * sum(TP_list)) / (2 * sum(TP_list) + sum(FNR_FPR)),
    )
    print("Combined F1: ", (2 * sum(TN_list)) / (2 * sum(TN_list) + sum(FNR_FPR)))


first = (0.15378071360656553, 0.900165686624866, 0.6654327708219924)
second = (0.21323102714552844, 0.8393334305008988, 0.6830857690464782)
third = (0.19794610241918809, 0.8222772277227722, 0.7766371130813501)

combine_f1_scores(first, 99913, second, 77544, third, 72543)

Combined accuracy:  0.8149636667386367
Combined alternative F1:  0.8629820730608789
Combined F1:  0.7151299847891536


In [13]:
# ()
X_train_zero, X_test_zero = clip_outliers(X_train_zero, X_test_zero)
X_train_one, X_test_one = clip_outliers(X_train_one, X_test_one)
X_train_many, X_test_many = clip_outliers(X_train_many, X_test_many)
X_train_full, X_test_full = clip_outliers(X_train_full, X_test_full)
print(X_train_zero.shape)
print(X_train_one.shape)
print(X_train_many.shape)
print(X_train_full.shape)

(24, 30)
(11, 30)
(15, 30)
(50, 30)


In [14]:
def missing_value_proportion(X):
    return np.count_nonzero(np.isnan(X), axis=0) / len(X)


def all_missing_indices(X):
    return X == 1


def no_missing_indices(X):
    return X == 0


def partial_missing_indices(X):
    return (X > 0) & (X < 1)

In [15]:
def feature_engg_X(X_train_set, X_test_set):
    """
    Performs some of our  feature engineering, replaces missing values and adds the binary encoding.
    """
    # replace empty value -999 by nan.
    X_train_set[X_train_set == -999] = np.nan
    X_test_set[X_test_set == -999] = np.nan

    # Get the missing value proportion for each column
    X_train_set_missing_proportion = missing_value_proportion(X_train_set)

    # X_train_set
    X_partial_missing = partial_missing_indices(X_train_set_missing_proportion)

    # Store new column to indicate whether the value is missing or not
    new_col_X_train = np.where(np.isnan(X_train_set[:, X_partial_missing]), 0, 1)
    new_col_X_test = np.where(np.isnan(X_test_set[:, X_partial_missing]), 0, 1)

    # fill missing value with median
    X_train_set_median = np.nanmedian(X_train_set[:, X_partial_missing])
    arr = X_train_set[:, X_partial_missing]
    arr[np.isnan(arr)] = X_train_set_median
    X_train_set[:, X_partial_missing] = arr

    arr = X_test_set[:, X_partial_missing]
    arr[np.isnan(arr)] = X_train_set_median
    X_test_set[:, X_partial_missing] = arr

    # drop the columns with all missing values for X_train and X_test
    X_zero_to_delete = all_missing_indices(X_train_set_missing_proportion)
    X_train_set = np.delete(X_train_set, X_zero_to_delete, axis=1)
    X_test_set = np.delete(X_test_set, X_zero_to_delete, axis=1)

    # Add the new column
    X_train_set = np.hstack([new_col_X_train, X_train_set])
    X_test_set = np.hstack([new_col_X_test, X_test_set])
    return X_train_set, X_test_set

In [16]:
X_train_zero, X_test_zero = feature_engg_X(X_train_zero, X_test_zero)
X_train_one, X_test_one = feature_engg_X(X_train_one, X_test_one)
X_train_many, X_test_many = feature_engg_X(X_train_many, X_test_many)

X_train_full, X_test_full = feature_engg_X(X_train_full, X_test_full)

c:\Users\lucas\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [17]:
def accuracy_loss(y, prediction):
    """Computes 1-accuracy of prediction, and also the F1-score"""
    # y values are either -1 or 1.
    # So if prediction and y are both negative -> it's correct or if prediction and y both are positive -> it's correct
    accuracy = ((y < 0.5) == (prediction < 0.5)).sum() / y.shape[0]
    TPR = ((y < 0.5) & (prediction < 0.5)).sum()
    FNR = ((y < 0.5) & (prediction >= 0.5)).sum()
    FPR = ((y >= 0.5) & (prediction < 0.5)).sum()
    TNR = ((y >= 0.5) & (prediction >= 0.5)).sum()
    # Depending on what one labels as a "positive" and what as a "negative" a different one of the F1 scores is relevant here.
    F_1_score = 2 * TPR / (2 * TPR + FNR + FPR)
    alt_F_1_score = 2 * TNR / (2 * TNR + FNR + FPR)
    return 1 - accuracy, F_1_score, alt_F_1_score


import itertools  # In standard library so should be allowed?


def tune_hyperparamters(
    model, parameter_ranges, train_x, train_y, test_x, test_y, metric=compute_loss_mse
):
    """
    Given a model and a list of parameters to that model and their ranges,
    for each combination of parameters this model will train them on the training dataset and then evaluate them on the test set.
    It returns the best parameters and their corresponding loss.
    Expects the model to take inputs in the following day: y,X, and then the parameters specified in the input in the same order.
    """
    best_loss = None
    best_param = None
    best_w = None
    for param in itertools.product(*parameter_ranges):
        w_vec, train_loss = model(train_y, train_x, *param)
        validation_loss = metric(test_y, test_x @ w_vec)
        if model in [logistic_regression, reg_logistic_regression]:
            prediction = sigmoid(test_x @ w_vec)
            validation_loss = metric(test_y, prediction)

        if best_loss == None or validation_loss < best_loss:
            best_loss, best_param = (validation_loss, param)
            best_w = w_vec
    return best_param, best_loss, best_w


def tune_model(
    model,
    parameter_ranges,
    poly_degree,
    dimension,
    train_x,
    train_y,
    test_x,
    test_y,
    metric=compute_loss_mse,
):
    """
    model: The model to optimize
    parameter_ranges: The input parameters to the model and the ranges to search for those
    poly_degree: List of degrees to use for polynomial expansion
    dimension: Dimension to project data down to using PCA
    train_x,train_y,test_x,test_y: Train and test data
    metric: Metric to use to compare. E.g. compute_loss_mse. Should expect two inputs: y and prediction
    Will perform an extensive hyperparameter search and return the ones that give the lowest value in the metric.
    Expects the model to take inputs in the following day: y,X, and then the parameters specified in the input in the same order.
    Return: best_param, best_loss, degree, dimension
    The best parameters in the parameter_ranges, the loss achieved for those, the degree of the polynomial expansion for the best value and the dimension
    """
    best_loss = None
    best_param = None
    best_degree = None
    best_dimension = None
    best_w = None
    for degree in poly_degree:
        poly_mapped_train = build_poly(train_x, degree)
        poly_mapped_test = build_poly(test_x, degree)

        preproc = Preprocessor()
        preproc.fit(poly_mapped_train)
        poly_mapped_train = preproc.standardize(poly_mapped_train)
        poly_mapped_test = preproc.standardize(poly_mapped_test)

        for pca_dimension in dimension:
            if pca_dimension > poly_mapped_train.shape[1]:
                continue
            mapped_train, _ = preproc.PCA(poly_mapped_train, pca_dimension)
            mapped_test, _ = preproc.PCA(poly_mapped_test, pca_dimension)

            # Specifiy a constant inital w such that it is of the right size for our adjusted data
            cur_param_range = parameter_ranges.copy()
            for i in range(0, len(parameter_ranges)):
                if parameter_ranges[i] == "initial_w":
                    cur_param_range[i] = [np.zeros(mapped_train.shape[1])]
            # Tune all other parameters
            param, cur_loss, w = tune_hyperparamters(
                model,
                cur_param_range,
                mapped_train,
                train_y,
                mapped_test,
                test_y,
                metric,
            )
            if best_loss == None or cur_loss < best_loss:
                best_loss, best_param = (cur_loss, param)
                best_degree, best_dimension = degree, pca_dimension
                best_w = w
    return best_param, best_loss, best_degree, best_dimension, best_w


def predict(X_test, ids, w, X_train, dimension, poly_degree, logistic=False):
    """
    Performs the model prediction.
    Given test dataset, the corresponding IDs, weight vector w, train dataset, PCA target dimension, polynomial degree, and a Boolean to indicate if we should use the sigmoid function or not, this generates the predictions and returns them.
    """
    poly_mapped_test = build_poly(X_test, poly_degree)
    poly_mapped_train = build_poly(X_train, poly_degree)
    preproc = Preprocessor()
    preproc.fit(poly_mapped_train)
    poly_mapped_train = preproc.standardize(poly_mapped_train)
    poly_mapped_test = preproc.standardize(poly_mapped_test)
    mapped_test, _ = preproc.PCA(poly_mapped_test, dimension)
    prediction = mapped_test @ w
    if logistic:
        prediction = sigmoid(prediction)
    for index in range(0, prediction.shape[0]):
        if prediction[index] < 0.5:
            prediction[index] = 0
        else:
            prediction[index] = 1
    return prediction

In [26]:
def evaluate_mse_gd(train_x, train_y, test_x, test_y, X_test):
    """
    Evaluates how well mse_gd works with various parameters
    """
    max_iters = [500]
    gamma = [0.001, 0.005, 0.01, 0.015]
    parameter_ranges = ["initial_w", max_iters, gamma]
    poly_degree = [2, 3, 4, 5, 6, 7, 8, 9, 10]
    dimension = [
        1,
        5,
        10,
        15,
        20,
        25,
        30,
        35,
        40,
        45,
        50,
        55,
        60,
        65,
        70,
        75,
        80,
        90,
        100,
        110,
        120,
        130,
        140,
        150,
        160,
        170,
        180,
        190,
        200,
        220,
        240,
        260,
        280,
        300,
    ]
    best_param, best_loss, best_degree, best_dimension, best_w = tune_model(
        mean_squared_error_gd,
        parameter_ranges,
        poly_degree,
        dimension,
        train_x,
        train_y,
        test_x,
        test_y,
        accuracy_loss,
    )
    print(best_param, best_loss, best_degree, best_dimension)
    accuracy = 1 - best_loss[0]
    f_1_score = best_loss[2]
    print(
        f"Optimum found, using degree {best_degree} and dimension {best_dimension} achieved accuracy of {accuracy} and F1 score of {f_1_score}"
    )
    prediction = predict(
        X_test, ids_test, best_w, train_x, best_dimension, best_degree, False
    )
    return prediction

In [27]:
def evaluate_mse_sgd(train_x, train_y, test_x, test_y, X_test):
    """
    Evaluates how well mse_sgd works with various parameters
    """
    max_iters = [50, 1000]
    gamma = [0.001, 0.0025, 0.005, 0.01, 0.025, 0.1]
    parameter_ranges = ["initial_w", max_iters, gamma]
    poly_degree = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
    dimension = [
        50,
        75,
        100,
        115,
        120,
        125,
        130,
        135,
        175,
        200,
        250,
        300,
        305,
        310,
        315,
        320,
        325,
        330,
        335,
        340,
        345,
        350,
    ]
    best_param, best_loss, best_degree, best_dimension, best_w = tune_model(
        mean_squared_error_sgd,
        parameter_ranges,
        poly_degree,
        dimension,
        train_x,
        train_y,
        test_x,
        test_y,
        accuracy_loss,
    )
    print(best_param, best_loss, best_degree, best_dimension)
    accuracy = 1 - best_loss[0]
    f_1_score = best_loss[2]
    print(
        f"Optimum found, using degree {best_degree} and dimension {best_dimension} achieved accuracy of {accuracy} and F1 score of {f_1_score}"
    )

    prediction = predict(
        X_test, ids_test, best_w, train_x, best_dimension, best_degree, False
    )
    return prediction

In [28]:
def evaluate_ridge(train_x, train_y, test_x, test_y, X_test):
    """
    Evaluates how well ridge regression works with various parameters
    """
    lambda_ = np.logspace(-20, 2, 150)
    parameter_ranges = [lambda_]
    poly_degree = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
    dimension = [
        50,
        75,
        100,
        115,
        120,
        125,
        130,
        135,
        175,
        200,
        250,
        300,
        305,
        310,
        315,
        320,
        325,
        330,
        335,
        340,
        345,
        350,
    ]
    best_param, best_loss, best_degree, best_dimension, best_w = tune_model(
        ridge_regression,
        parameter_ranges,
        poly_degree,
        dimension,
        train_x,
        train_y,
        test_x,
        test_y,
        accuracy_loss,
    )
    print(best_param, best_loss, best_degree, best_dimension)
    accuracy = 1 - best_loss[0]
    f_1_score = best_loss[2]
    print(
        f"Optimum found, using degree {best_degree} and dimension {best_dimension} achieved accuracy of {accuracy} and F1 score of {f_1_score}"
    )
    prediction = predict(
        X_test, ids_test, best_w, train_x, best_dimension, best_degree, False
    )
    return prediction

In [29]:
def evaluate_log_regr(train_x, train_y, test_x, test_y, X_test):
    """
    Evaluates how well ridge regression works with various parameters
    """
    max_iters = [500]
    gamma = [0.0001, 0.001, 0.01, 0.1, 1]
    parameter_ranges = ["initial_w", max_iters, gamma]
    poly_degree = [2, 3, 4, 5, 6, 7, 8, 9, 10]
    dimension = [
        1,
        5,
        10,
        15,
        20,
        25,
        30,
        35,
        40,
        45,
        50,
        55,
        60,
        65,
        70,
        75,
        80,
        90,
        100,
        110,
        120,
        130,
        140,
        150,
        160,
        170,
        180,
        190,
        200,
        220,
        240,
        260,
        280,
        300,
    ]
    best_param, best_loss, best_degree, best_dimension, best_w = tune_model(
        logistic_regression,
        parameter_ranges,
        poly_degree,
        dimension,
        train_x,
        train_y,
        test_x,
        test_y,
        accuracy_loss,
    )
    print(best_param, best_loss, best_degree, best_dimension)
    accuracy = 1 - best_loss[0]
    f_1_score = best_loss[2]
    print(
        f"Optimum found, using degree {best_degree} and dimension {best_dimension} achieved accuracy of {accuracy} and F1 score of {f_1_score}"
    )

    return predict(X_test, ids_test, best_w, train_x, best_dimension, best_degree, True)

In [30]:
def evaluate_reg_log_regr(train_x, train_y, test_x, test_y, X_test):
    """
    Evaluates how well ridge regression works with various parameters
    """
    max_iters = [500]
    gamma = [0.001, 0.05, 0.1, 0.15]
    lambda_ = np.logspace(-10, 2, 25)
    parameter_ranges = [lambda_, "initial_w", max_iters, gamma]
    poly_degree = [3, 4, 5, 6, 7]
    dimension = [
        1,
        5,
        10,
        15,
        20,
        25,
        30,
        35,
        40,
        45,
        50,
        55,
        60,
        65,
        70,
        75,
        80,
        90,
        100,
        110,
        120,
        130,
        140,
        150,
        160,
        170,
        180,
        190,
        200,
        220,
        240,
        260,
        280,
        300,
    ]
    best_param, best_loss, best_degree, best_dimension, best_w = tune_model(
        reg_logistic_regression,
        parameter_ranges,
        poly_degree,
        dimension,
        train_x,
        train_y,
        test_x,
        test_y,
        accuracy_loss,
    )
    print(best_param, best_loss, best_degree, best_dimension)
    accuracy = 1 - best_loss[0]
    f_1_score = best_loss[2]
    print(
        f"Optimum found, using degree {best_degree} and dimension {best_dimension} achieved accuracy of {accuracy} and F1 score of {f_1_score}"
    )
    return predict(X_test, ids_test, best_w, train_x, best_dimension, best_degree, True)

In [31]:
def evaluate_least_squares(train_x, train_y, test_x, test_y, X_test):
    """
    Evaluates how well least squares works with various parameters
    """
    parameter_ranges = []
    poly_degree = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
    dimension = [
        50,
        75,
        100,
        115,
        120,
        125,
        130,
        135,
        175,
        200,
        250,
        300,
        305,
        310,
        315,
        320,
        325,
        330,
        335,
        340,
        345,
        350,
    ]
    best_param, best_loss, best_degree, best_dimension, best_w = tune_model(
        least_squares,
        parameter_ranges,
        poly_degree,
        dimension,
        train_x,
        train_y,
        test_x,
        test_y,
        accuracy_loss,
    )
    print(best_param, best_loss, best_degree, best_dimension)
    accuracy = 1 - best_loss[0]
    f_1_score = best_loss[2]
    print(
        f"Optimum found, using degree {best_degree} and dimension {best_dimension} achieved accuracy of {accuracy} and F1 score of {f_1_score}"
    )

    prediction = predict(
        X_test, ids_test, best_w, train_x, best_dimension, best_degree, False
    )
    return prediction

In [32]:
def pred_each_jet_set(X_train, y_train, X_test, model_evaluate=evaluate_least_squares):
    """
    Given a single subset of data, trains the model and generates the predictions from that.
    """

    X_train, X_val, y_train, y_val = split_data(X_train, y_train, 0.8, 123)

    pred = model_evaluate(X_train, y_train, X_val, y_val, X_test)
    return pred


def pred_all_sets(
    X_train_list,
    y_train_list,
    X_test_list,
    id_list,
    model_to_evaluate=evaluate_least_squares,
    output_name="prediction.csv",
):
    """
    Arguments:
    X_train_list: A list of our X_train sets
    y_train_list: A list of our output values for the train sets
    X_test_list: A list of outputs to predict using our trained model
    id_list: A list of the output IDs for the tests.
    model_to_evaluate: One of the evaluate_* functions to use to evaluate the model
    output_name: Name of output file to write
    """
    labels = []
    for (X_train, y_train, X_test) in zip(X_train_list, y_train_list, X_test_list):
        labels.append(pred_each_jet_set(X_train, y_train, X_test, model_to_evaluate))
    label_all = np.hstack(labels)
    ids_test = np.hstack(id_list)
    create_csv_submission(ids_test, label_all, output_name)

In [33]:
pred_all_sets(
    [X_train_zero, X_train_one, X_train_many],
    [y_train_zero, y_train_one, y_train_many],
    [X_test_zero, X_test_one, X_test_many],
    [ids_test_zero, ids_test_one, ids_test_many],
    evaluate_mse_gd,
    "mse_gd.csv",
)

(array([0., 0., 0., 0., 0.]), 500, 0.01) (0.19999999999999996, 0.8, 0.8) 8 5
Optimum found, using degree 8 and dimension 5 achieved accuracy of 0.8 and F1 score of 0.8
(array([0., 0., 0., 0., 0.]), 500, 0.01) (0.33333333333333337, 0.6666666666666666, 0.6666666666666666) 3 5
Optimum found, using degree 3 and dimension 5 achieved accuracy of 0.6666666666666666 and F1 score of 0.6666666666666666


c:\Users\lucas\Documents\Actual_Documents\ETH\Sem_3\machine_learninig\project1\ml-project-1-luminous-hack\implementations.py:199: RuntimeWarning: invalid value encountered in subtract
  w = w - grad * gamma
C:\Users\lucas\AppData\Local\Temp\ipykernel_29932\3540802968.py:11: RuntimeWarning: invalid value encountered in long_scalars
  F_1_score = 2 * TPR / (2 * TPR + FNR + FPR)
C:\Users\lucas\AppData\Local\Temp\ipykernel_29932\3540802968.py:12: RuntimeWarning: invalid value encountered in long_scalars
  alt_F_1_score = 2 * TNR / (2 * TNR + FNR + FPR)


(array([0.]), 500, 0.001) (0.0, 1.0, 1.0) 2 1
Optimum found, using degree 2 and dimension 1 achieved accuracy of 1.0 and F1 score of 1.0


In [34]:
pred_all_sets(
    [X_train_zero, X_train_one, X_train_many],
    [y_train_zero, y_train_one, y_train_many],
    [X_test_zero, X_test_one, X_test_many],
    [ids_test_zero, ids_test_one, ids_test_many],
    evaluate_least_squares,
    "lsqs.csv",
)

c:\Users\lucas\Documents\Actual_Documents\ETH\Sem_3\machine_learninig\project1\ml-project-1-luminous-hack\implementations.py:247: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  w = np.linalg.lstsq(tx, y)[0]


In [ ]:
pred_all_sets(
    [X_train_zero, X_train_one, X_train_many],
    [y_train_zero, y_train_one, y_train_many],
    [X_test_zero, X_test_one, X_test_many],
    [ids_test_zero, ids_test_one, ids_test_many],
    evaluate_mse_sgd,
    "mse_sgd.csv",
)

In [ ]:
pred_all_sets(
    [X_train_zero, X_train_one, X_train_many],
    [y_train_zero, y_train_one, y_train_many],
    [X_test_zero, X_test_one, X_test_many],
    [ids_test_zero, ids_test_one, ids_test_many],
    evaluate_ridge,
    "ridge.csv",
)

In [ ]:
pred_all_sets(
    [X_train_zero, X_train_one, X_train_many],
    [y_train_zero, y_train_one, y_train_many],
    [X_test_zero, X_test_one, X_test_many],
    [ids_test_zero, ids_test_one, ids_test_many],
    evaluate_log_regr,
    "log_regr.csv",
)

In [ ]:
pred_all_sets(
    [X_train_zero, X_train_one, X_train_many],
    [y_train_zero, y_train_one, y_train_many],
    [X_test_zero, X_test_one, X_test_many],
    [ids_test_zero, ids_test_one, ids_test_many],
    evaluate_reg_log_regr,
    "reg_log_regr.csv",
)

# Now we do it all for the combined dataset:

In [ ]:
pred_all_sets(
    [X_train_zero, X_train_one, X_train_many],
    [y_train_zero, y_train_one, y_train_many],
    [X_test_zero, X_test_one, X_test_many],
    [ids_test_zero, ids_test_one, ids_test_many],
    evaluate_mse_gd,
    "combined_mse_gd.csv",
)
pred_all_sets(
    [X_train_zero, X_train_one, X_train_many],
    [y_train_zero, y_train_one, y_train_many],
    [X_test_zero, X_test_one, X_test_many],
    [ids_test_zero, ids_test_one, ids_test_many],
    evaluate_least_squares,
    "combined_lsqs.csv",
)
pred_all_sets(
    [X_train_zero, X_train_one, X_train_many],
    [y_train_zero, y_train_one, y_train_many],
    [X_test_zero, X_test_one, X_test_many],
    [ids_test_zero, ids_test_one, ids_test_many],
    evaluate_mse_sgd,
    "combined_mse_sgd.csv",
)
pred_all_sets(
    [X_train_zero, X_train_one, X_train_many],
    [y_train_zero, y_train_one, y_train_many],
    [X_test_zero, X_test_one, X_test_many],
    [ids_test_zero, ids_test_one, ids_test_many],
    evaluate_ridge,
    "combined_ridge.csv",
)
pred_all_sets(
    [X_train_zero, X_train_one, X_train_many],
    [y_train_zero, y_train_one, y_train_many],
    [X_test_zero, X_test_one, X_test_many],
    [ids_test_zero, ids_test_one, ids_test_many],
    evaluate_log_regr,
    "combined_log_regr.csv",
)
pred_all_sets(
    [X_train_zero, X_train_one, X_train_many],
    [y_train_zero, y_train_one, y_train_many],
    [X_test_zero, X_test_one, X_test_many],
    [ids_test_zero, ids_test_one, ids_test_many],
    evaluate_reg_log_regr,
    "combined_reg_log_regr.csv",
)